In [1]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [159]:
#計測地点のosm_idを指定
start_osm = 668319081
mid_osm = [385509583]
end_osm = 385509577
start_mesh = 50325641322
end_mesh = 50325633332

In [160]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [161]:
getdbname = "kddi_18_new_param"
dbname = "kddi_trkn18_monthly_anbun_4b_32"
trknName = "trkn_18_grp"
sttime = "2022-04-01 18:00:00"

In [162]:
datedir = "230225_1"
f = open(datedir+'/'+ dbname + '.csv', 'w')
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        #cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        #cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        #cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            #cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            #cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        #cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        #print(sql)
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')
        trknStParam

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        #cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                #print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                #cur.execute(sql)
                #connection.commit()

        #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
        #cur.execute(sql)
        #connection.commit()
        #cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)
f.close()

33.7817708335:33.7880208335:132.767968750:132.767968750
存在メッシュ：50325641322値64.2距離0.0 km
50325641144 is kddi none!
存在メッシュ：50325642232値77.4652370368054距離1.4654394124113244 km
存在メッシュ：50325642221値79.20439729477087距離1.657568205847179 km
存在メッシュ：50325642233値76.03558009045362距離1.3075021188975637 km
存在メッシュ：50325642322値74.6792956618142距離1.1576704460331162 km
存在メッシュ：50325642223値78.94694659191282距離1.6291270701422988 km
存在メッシュ：50325642411値75.98920761759943距離1.3023792515727701 km
存在メッシュ：50325642214値77.66988748582978距離1.488047589930948 km
存在メッシュ：50325642234値77.34088536488474距離1.4517020143892478 km
存在メッシュ：50325632443値79.52902955059157距離1.693431032941764 km
存在メッシュ：50325642231値76.1734793100797距離1.3227361438019158 km
50325642321 is kddi none!
存在メッシュ：50325633331値82.34887802422607距離2.004945789807886 km
存在メッシュ：50325633332値83.58333333333334距離2.141318736465901 km
存在メッシュ：50325632442値81.1257769065505距離1.8698271652229634 km
存在メッシュ：50325642221値79.20439729477087距離1.657568205847179 km
存在メッシュ：50325632443値79.52902955

33.7817708335:33.7880208335:132.767968750:132.767968750
存在メッシュ：50325641322値38.3距離0.0 km
50325641144 is kddi none!
存在メッシュ：50325642232値49.534960001077664距離1.4654394124113244 km
存在メッシュ：50325642221値51.007937519646866距離1.657568205847179 km
存在メッシュ：50325642233値48.32411555382356距離1.3075021188975637 km
存在メッシュ：50325642322値47.17541378064229距離1.1576704460331162 km
存在メッシュ：50325642223値50.789890277759355距離1.6291270701422988 km
存在メッシュ：50325642411値48.28484050157819距離1.3023792515727701 km
存在メッシュ：50325642214値49.70828819737088距離1.488047589930948 km
存在メッシュ：50325642234値49.429640657275556距離1.4517020143892478 km
存在メッシュ：50325632443値51.28288401318374距離1.693431032941764 km
存在メッシュ：50325642231値48.44090895995571距離1.3227361438019158 km
50325642321 is kddi none!
存在メッシュ：50325633331値53.6711477677237距離2.004945789807886 km
存在メッシュ：50325633332値54.71666666666666距離2.141318736465901 km
存在メッシュ：50325632442値52.635245273389714距離1.8698271652229634 km
存在メッシュ：50325642221値51.007937519646866距離1.657568205847179 km
存在メッシュ：50325632443値51

33.7817708335:33.7880208335:132.767968750:132.767968750
存在メッシュ：50325641322値18.916666666666668距離0.0 km
存在メッシュ：50325641144値19.267390390224822距離0.11554019701202542 km
存在メッシュ：50325642232値23.365026464047673距離1.4654394124113244 km
存在メッシュ：50325642221値23.948235836882183距離1.657568205847179 km
存在メッシュ：50325642233値22.88560581995722距離1.3075021188975637 km
50325642322 is kddi none!
存在メッシュ：50325642223値23.861902411160223距離1.6291270701422988 km
存在メッシュ：50325642411値22.87005529165702距離1.3023792515727701 km
存在メッシュ：50325642214値23.43365387171707距離1.488047589930948 km
存在メッシュ：50325642234値23.32332641929354距離1.4517020143892478 km
存在メッシュ：50325632443値24.057097900313025距離1.693431032941764 km
存在メッシュ：50325642231値22.931848894466395距離1.3227361438019158 km
50325642321 is kddi none!
存在メッシュ：50325633331値25.00270486911565距離2.004945789807886 km
存在メッシュ：50325633332値25.41666666666667距離2.141318736465901 km
存在メッシュ：50325632442値24.592550582018944距離1.8698271652229634 km
存在メッシュ：50325642221値23.948235836882183距離1.657568205847179 km
存在メ

33.7817708335:33.7880208335:132.767968750:132.767968750
存在メッシュ：50325641322値7.271186440677966距離0.0 km
50325641144 is kddi none!
存在メッシュ：50325642232値9.042217475176333距離1.4654394124113244 km
存在メッシュ：50325642221値9.274411350210531距離1.657568205847179 km
存在メッシュ：50325642233値8.851345136171002距離1.3075021188975637 km
存在メッシュ：50325642322値8.670268702477227距離1.1576704460331162 km
存在メッシュ：50325642223値9.240039315745564距離1.6291270701422988 km
存在メッシュ：50325642411値8.845153984942922距離1.3023792515727701 km
存在メッシュ：50325642214値9.069540190589601距離1.488047589930948 km
存在メッシュ：50325642234値9.025615384346578距離1.4517020143892478 km
存在メッシュ：50325632443値9.317752739306139距離1.693431032941764 km
存在メッシュ：50325642231値8.869755949103578距離1.3227361438019158 km
50325642321 is kddi none!
存在メッシュ：50325633331値9.694228421235772距離2.004945789807886 km
存在メッシュ：50325633332値9.859039548022599距離2.141318736465901 km
50325632442 is kddi none!
存在メッシュ：50325642221値9.274411350210531距離1.657568205847179 km
存在メッシュ：50325632443値9.317752739306139距離1.6934310

33.7817708335:33.7880208335:132.767968750:132.767968750
存在メッシュ：50325641322値44.05距離0.0 km
存在メッシュ：50325641144値44.86295960537582距離0.11554019701202542 km
存在メッシュ：50325642232値54.36106988931392距離1.4654394124113244 km
存在メッシュ：50325642221値55.71291930737134距離1.657568205847179 km
存在メッシュ：50325642233値53.249797421986294距離1.3075021188975637 km
存在メッシュ：50325642322値52.1955574189854距離1.1576704460331162 km
存在メッシュ：50325642223値55.512802853903距離1.6291270701422988 km
存在メッシュ：50325642411値53.213752094849426距離1.3023792515727701 km
存在メッシュ：50325642214値54.52014470093733距離1.488047589930948 km
存在メッシュ：50325642234値54.26441132403767距離1.4517020143892478 km
存在メッシュ：50325632443値55.96525598773411距離1.693431032941764 km
存在メッシュ：50325642231値53.356986497258845距離1.3227361438019158 km
50325642321 is kddi none!
存在メッシュ：50325633331値58.15712444875353距離2.004945789807886 km
50325633332 is kddi none!
存在メッシュ：50325632442値57.206407844816546距離1.8698271652229634 km
存在メッシュ：50325642221値55.71291930737134距離1.657568205847179 km
存在メッシュ：50325632443値55.

...

33.7817708335:33.7880208335:132.767968750:132.767968750
存在メッシュ：50325641322値18.916666666666668距離0.0 km
存在メッシュ：50325641144値19.267390390224822距離0.11554019701202542 km
存在メッシュ：50325642232値23.365026464047673距離1.4654394124113244 km
存在メッシュ：50325642221値23.948235836882183距離1.657568205847179 km
存在メッシュ：50325642233値22.88560581995722距離1.3075021188975637 km
存在メッシュ：50325642322値22.430789889459046距離1.1576704460331162 km
存在メッシュ：50325642223値23.861902411160223距離1.6291270701422988 km
存在メッシュ：50325642411値22.87005529165702距離1.3023792515727701 km
存在メッシュ：50325642214値23.43365387171707距離1.488047589930948 km
存在メッシュ：50325642234値23.32332641929354距離1.4517020143892478 km
存在メッシュ：50325632443値24.057097900313025距離1.693431032941764 km
存在メッシュ：50325642231値22.931848894466395距離1.3227361438019158 km
50325642321 is kddi none!
存在メッシュ：50325633331値25.00270486911565距離2.004945789807886 km
存在メッシュ：50325633332値25.41666666666667距離2.141318736465901 km
存在メッシュ：50325632442値24.592550582018944距離1.8698271652229634 km
存在メッシュ：50325642221値23.948235

存在メッシュ：50325641322値7.271186440677966距離0.0 km
50325641144 is kddi none!
存在メッシュ：50325642232値9.042217475176333距離1.4654394124113244 km
存在メッシュ：50325642221値9.274411350210531距離1.657568205847179 km
存在メッシュ：50325642233値8.851345136171002距離1.3075021188975637 km
存在メッシュ：50325642322値8.670268702477227距離1.1576704460331162 km
存在メッシュ：50325642223値9.240039315745564距離1.6291270701422988 km
存在メッシュ：50325642411値8.845153984942922距離1.3023792515727701 km
存在メッシュ：50325642214値9.069540190589601距離1.488047589930948 km
存在メッシュ：50325642234値9.025615384346578距離1.4517020143892478 km
存在メッシュ：50325632443値9.317752739306139距離1.693431032941764 km
存在メッシュ：50325642231値8.869755949103578距離1.3227361438019158 km
50325642321 is kddi none!
存在メッシュ：50325633331値9.694228421235772距離2.004945789807886 km
存在メッシュ：50325633332値9.859039548022599距離2.141318736465901 km
存在メッシュ：50325632442値9.530933183335167距離1.8698271652229634 km
存在メッシュ：50325642221値9.274411350210531距離1.657568205847179 km
存在メッシュ：50325632443値9.317752739306139距離1.693431032941764 km
存在メッシュ：503

33.7817708335:33.7880208335:132.767968750:132.767968750
50325641322 is kddi none!
存在メッシュ：50325641144値44.86295960537582距離0.11554019701202542 km
存在メッシュ：50325642232値54.36106988931392距離1.4654394124113244 km
存在メッシュ：50325642221値55.71291930737134距離1.657568205847179 km
存在メッシュ：50325642233値53.249797421986294距離1.3075021188975637 km
存在メッシュ：50325642322値52.1955574189854距離1.1576704460331162 km
存在メッシュ：50325642223値55.512802853903距離1.6291270701422988 km
存在メッシュ：50325642411値53.213752094849426距離1.3023792515727701 km
存在メッシュ：50325642214値54.52014470093733距離1.488047589930948 km
存在メッシュ：50325642234値54.26441132403767距離1.4517020143892478 km
存在メッシュ：50325632443値55.96525598773411距離1.693431032941764 km
存在メッシュ：50325642231値53.356986497258845距離1.3227361438019158 km
50325642321 is kddi none!
存在メッシュ：50325633331値58.15712444875353距離2.004945789807886 km
存在メッシュ：50325633332値59.11666666666666距離2.141318736465901 km
存在メッシュ：50325632442値57.206407844816546距離1.8698271652229634 km
存在メッシュ：50325642221値55.71291930737134距離1.657568205847179

33.7817708335:33.7880208335:132.767968750:132.767968750
存在メッシュ：50325641322値19.1距離0.0 km
存在メッシュ：50325641144値19.560437298619938距離0.11554019701202542 km
存在メッシュ：50325642232値24.939897990407886距離1.4654394124113244 km
存在メッシュ：50325642221値25.705547218334214距離1.657568205847179 km
存在メッシュ：50325642233値24.310504734576313距離1.3075021188975637 km
存在メッシュ：50325642322値23.713413051460766距離1.1576704460331162 km
存在メッシュ：50325642223値25.592206926104357距離1.6291270701422988 km
存在メッシュ：50325642411値24.29008968203862距離1.3023792515727701 km
存在メッシュ：50325642214値25.029993458937962距離1.488047589930948 km
存在メッシュ：50325642234値24.885153316269125距離1.4517020143892478 km
存在メッシュ：50325632443値25.84846356827419距離1.693431032941764 km
存在メッシュ：50325642231値24.371213591367848距離1.3227361438019158 km
50325642321 is kddi none!
存在メッシュ：50325633331値27.089875793984305距離2.004945789807886 km
存在メッシュ：50325633332値27.633333333333336距離2.141318736465901 km
存在メッシュ：50325632442値26.551416832462476距離1.8698271652229634 km
存在メッシュ：50325642221値25.705547218334214距

33.7817708335:33.7880208335:132.767968750:132.767968750
存在メッシュ：50325641322値8.932203389830509距離0.0 km
50325641144 is kddi none!
存在メッシュ：50325642232値10.96170713398113距離1.4654394124113244 km
存在メッシュ：50325642221値11.22778848974022距離1.657568205847179 km
存在メッシュ：50325642233値10.74297797615142距離1.3075021188975637 km
存在メッシュ：50325642322値10.535474384740304距離1.1576704460331162 km
存在メッシュ：50325642223値11.188400036753295距離1.6291270701422988 km
存在メッシュ：50325642411値10.735883258806984距離1.3023792515727701 km
存在メッシュ：50325642214値10.993017456697917距離1.488047589930948 km
存在メッシュ：50325642234値10.94268205489252距離1.4517020143892478 km
存在メッシュ：50325632443値11.277455327719228距離1.693431032941764 km
存在メッシュ：50325642231値10.764075750866647距離1.3227361438019158 km
50325642321 is kddi none!
存在メッシュ：50325633331値11.70887566489828距離2.004945789807886 km
存在メッシュ：50325633332値11.89774011299435距離2.141318736465901 km
存在メッシュ：50325632442値11.521748341736991距離1.8698271652229634 km
存在メッシュ：50325642221値11.22778848974022距離1.657568205847179 km
存在メッシュ

In [163]:
connection.close()

In [133]:
#CreateGeojsonFile